# Exploration of Passage Point API

In [ ]:
import requests
from datetime import datetime

Add our API credentials for our instance. 

In [ ]:
username = ''
password = '' 
instance = '' #PassagePoint server:port
barcode = '' # e.g. 22222222222222

In [ ]:
credentials_endpt = f'{instance}/pp/api/v2/auth/login'
uniqueid_endpt = f'{instance}/pp/api/v2/person/byUniqueId'
create_visitor_endpt = f'{instance}/pp/api/v2/person/createVisitor'
create_prereg_endpt = f'{instance}/pp/api/v2/visit/createPreReg'
get_destinations_endpt = f'{instance}/pp/api/v2/visit/getDestinations'

In [ ]:
def create_cred_body(username: str, password: str):
    '''Creates the body of a POST request for obtaining an access token, given a username and password.'''
    return {'username': username,
           'password': password}

In [ ]:
def get_token(cred_body: dict):
    '''Retrieves an access token from the PassagePoint login API, given a request body as specified.'''
    try:
        resp = requests.post(credentials_endpt, json=cred_body)
        resp.raise_for_status()
        token_data = resp.json()
        if 'error' in token_data:
            raise Exception(token_data)
        return token_data["token"]
    except Exception as e:
        print(e)
        raise

Example of obtaining a token for further calls. 

In [ ]:
cred_body = create_cred_body(username, password)
token = get_token(cred_body)

In [ ]:
token

In [ ]:
def create_header(token: str):
    '''Creates a header with the supplied token from the login API.'''
    try:
        return {'token': token, 'Content Type': 'application/json'}
    except Exception as e:
        print('Error', e)

In [ ]:
def create_visitor(header: dict, visitor: dict):
    '''Sends a POST request to create a new visitor using a uniqueId'''
    params = {'category': 'Visitor',
              'firstName': visitor['firstName'],
              'lastName': visitor['lastName'],
              'uniqueId': barcode} 
    try:
        resp = requests.post(create_visitor_endpt, headers=header, params=params)
        resp.raise_for_status()
        visitor_data = resp.json()
        if 'error' in visitor_data:
            raise Exception(visitor_data)
        return visitor_data
    except Exception as e:
        print('Error getting visitor.', e)
        raise

Create an example new visitor, providing their barcode in the request.

In [ ]:
header = create_header(token)
example_visitor = {'firstName': 'Test4', 'lastName': 'Patron4', 'uniqueId': barcode}
new_visitor = create_visitor(header, example_visitor)
print(new_visitor)

### Look up the same visitor by barcode number, using the uniqueId endpoint. 

Note that the PP API docs show the response as an array of users. If it is indeed possible that there are multiple users who match a uniqueId field, that is important to be aware of. 

In [ ]:
def get_visitor_bybarcode(header: dict, barcode: str):
    '''Retrieves the visitor ID from PassagePoint for a provided barcode in the visitor's unique ID field.'''
    try:
        params = {'uniqueId': barcode}
        resp = requests.get(uniqueid_endpt, headers=header, params=params)
        resp.raise_for_status()
        visitor_data = resp.json()
        if (not visitor_data) or ('id' not in visitor_data[0]):
            raise Exception(visitor_data)
        if len(visitor_data) > 1: # TODO: consider what do when multiple users returned
            raise Exception(visitor_data)
        visitor_id = str(visitor_data[0]['id'])
        return visitor_id
    except Exception as e:
        print('Error getting visitor.', e)
        raise
  

In [ ]:
barcode = '22882063227958'

In [ ]:
header = create_header(token)
visitor = get_visitor_bybarcode(header, barcode)
visitor

In [ ]:
def create_prereg(header: dict, booking: dict, visitor: str):
    '''Creates a Pre-Registration with the visitor ID and LibCal data.
    Requires a booking dict with a visitorId as int, startTime and endTime
    '''
    try:
        format_str = '%Y-%m-%dT%H:%M:%S%z'
        booking["startTime"] = str(int(datetime.strptime(booking['startTime'], format_str).timestamp()))
        booking["endTime"] = str(int(datetime.strptime(booking['endTime'], format_str).timestamp()))
        booking["visitorId"] = visitor
        booking["destination"] = "LibCal" # needs to exist in PP
        resp = requests.post(create_prereg_endpt, headers=header, json=booking)
        resp.raise_for_status()
        prereg_data = resp.json()
        if 'error' in prereg_data:
            raise Exception(prereg_data)
        return prereg_data
    except Exception as e:
        print('Error creating pre-registration.', e)
        raise

In [ ]:
test_booking = {"startTime": "2020-08-14T20:05:00-04:00", "endTime": "2020-08-14T22:05:00-04:00"}
header = create_header(token)
reg = create_prereg(header, test_booking, visitor)
print(reg)

In [ ]:
def get_destinations(header: dict):
    '''Retrieves the destinations from PassagePoint and returns as dict'''
    try:
        resp = requests.get(get_destinations_endpt, headers=header)
        resp.raise_for_status()
        destinations = resp.json()
        return destinations
    except Exception as e:
        print('Error getting visitor.', e)
        raise
  

In [ ]:
d = get_destinations(header)
print(d)

In [ ]:
# Testing Visitor record

In [ ]:
visitor_id = '137505764541148'
